In [4]:
!pip3 install numpy pandas matplotlib sparqlwrapper

Collecte de données

In [8]:
import requests
import glob
import shutil
import os
import sys
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

# Get cities
query = """SELECT DISTINCT ?tour ?tourLabel ?image {
  ?tour wdt:P31 wd:Q12518;
          wdt:P18 ?image.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "fr". }
}
LIMIT 100"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (
        sys.version_info[0],
        sys.version_info[1],
    )
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


array = []
results = get_results(endpoint_url, query)

for result in results["results"]["bindings"]:
    array.append(
        (
            result["tourLabel"]["value"],
            result["image"]["value"],
        )
    )

dataframe = pd.DataFrame(array, columns=["tour", "image"])
dataframe = dataframe.astype(
    dtype={"tour": "<U200", "image": "<U200"}
)



def download_image(url):
    headers = {"User-Agent": "Mozilla/5.0"}
    request = requests.get(url, allow_redirects=True, headers=headers, stream=True)
    if request.status_code == 200:
        with open(os.path.join("images", os.path.basename(url)), "wb") as image:
            request.raw.decode_content = True
            shutil.copyfileobj(request.raw, image)
    return request.status_code

#dataframe.image.apply(download_image)

Métadonnées des images

In [9]:
import json
import os
from PIL import Image
from PIL.ExifTags import TAGS

def save_metadata(directory, output_file="image_metadata.json"):
    metadata_data = {}

    for filename in os.listdir(directory):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
            image_path = os.path.join(directory, filename)
            imgfile = Image.open(image_path)
            exif_data = imgfile._getexif()

            metadata = {}
            metadata["image_path"] = image_path
            metadata["size"] = imgfile.size
            metadata["format"] = imgfile.format
            metadata["mode"] = imgfile.mode

            if exif_data:
                for tag, value in exif_data.items():
                    if tag in TAGS:
                        metadata[TAGS[tag]] = value

            metadata_data[image_path] = metadata

    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(metadata_data, f, ensure_ascii=False, indent=2)

# Exemple d'utilisation
directory = "./images"
save_metadata(directory)

TypeError: Object of type bytes is not JSON serializable

In [6]:
import os
import json
from PIL import Image
from PIL.ExifTags import TAGS
import glob
from fractions import Fraction
from PIL.TiffTags import TAGS as TIFFTAGS

def get_exif_data(img):
    exif_data = img._getexif()
    exif = {}
    if exif_data:
        for tag, value in exif_data.items():
            if tag in TAGS:
                if isinstance(value, bytes):
                    value = value.decode(errors='ignore')
                elif isinstance(value, (Fraction, int, float)):
                    value = str(value)
                elif isinstance(value, tuple):
                    value = tuple(str(v) for v in value)
                else :
                    value = str(value)
                exif[TAGS[tag]] = value
    return exif

def get_image_metadata(image_path):
    img = Image.open(image_path)
    exif_data = get_exif_data(img)
    exif_data.pop('MakerNote', None)
    metadata = {
        "filename": os.path.basename(image_path),
        "format": img.format,
        "size": img.size,
        "mode": img.mode,
        "orientation": "landscape" if img.width > img.height else "portrait",
        "exif": exif_data
    }
    return metadata

def save_metadata_to_json(metadata, output_file="image_tags.json"):
    try:
        with open(output_file, 'r') as f:
            data = json.load(f)
    except (FileNotFoundError, json.JSONDecodeError):
        data = []

    data.append(metadata)

    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

image_files = glob.glob("images/*.jpg")
for image_path in image_files:
    metadata = get_image_metadata(image_path)
    save_metadata_to_json(metadata)
    print(f"Métadonnées de l'image {image_path} enregistrées.")

Métadonnées de l'image images/Sender%20Bad%20Mergentheim%20%28Drillberg%29.jpg enregistrées.
Métadonnées de l'image images/Sender%20Bovenden.jpg enregistrées.
Métadonnées de l'image images/Sender%20Vogtsburg-Totenkopf.jpg enregistrées.
Métadonnées de l'image images/Swkrzyzwieza.jpg enregistrées.
Métadonnées de l'image images/Turm%202%20Koenigstuhl%20Heidelberg.jpg enregistrées.
Métadonnées de l'image images/TV-toren%20Goes.jpg enregistrées.
Métadonnées de l'image images/WAK%20THAL%20SCHARFENBURG7.jpg enregistrées.
Métadonnées de l'image images/Waldenburg%20Fernsehturm02%202007-09-22.jpg enregistrées.
Métadonnées de l'image images/Fernmeldeturm%20Hofkirchen.jpg enregistrées.
Métadonnées de l'image images/Fernmeldeturm%20Hohe%20Wurzel%20%28Wiesbaden%29.jpg enregistrées.
Métadonnées de l'image images/Fernmeldeturm%20Kahlheid.jpg enregistrées.
Métadonnées de l'image images/Fernmeldeturm%20Landau.jpg enregistrées.
Métadonnées de l'image images/Fernmeldeturm%20Lange%20Sicht.jpg enregistrées.